In [1]:
from __future__ import print_function
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os 
import glob
import skimage.io as io
import skimage.transform as trans
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras import backend as K
from unet_model import unet,dice_coef,dice_coef_loss
from data import adjustData,trainGenerator,testGenerator,labelVisualize,saveResult
from tensorflow.keras.callbacks import  EarlyStopping, ReduceLROnPlateau 
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import cv2
import os
from PIL import Image

X_test = []
for filepath in os.listdir('../data/test_images/ventral_samples_R0004'):
    image = Image.open('../data/test_images/ventral_samples_R0004/'+filepath)
    image = image.resize((256, 256))
    # convert image to numpy array
    data = np.asarray(image)
    data = data/255.
    X_test.append(data)
X_test = np.array(X_test)
Y_test = []
for filepath in os.listdir('../data/test_images/ventral_mask_atrium_R0004_binary'):
    image = Image.open('../data/test_images/ventral_mask_atrium_R0004_binary/'+filepath).convert('L')
    image = image.resize((256, 256))
    data = np.asarray(image).reshape(256,256,1)
    data = data/255.
    Y_test.append(data)
Y_test = np.array(Y_test)

In [3]:
from keras.preprocessing.image import ImageDataGenerator

batch_size = 10
epochs = 400
iterations_per_epoch = 300

data_gen_args = dict(rotation_range=0.3,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.1,
                    horizontal_flip=True,
                    fill_mode='nearest')

train_generator = trainGenerator(batch_size,'../data/train_images','ventral_samples','ventral_mask_atrium_binary',data_gen_args,image_color_mode='rgb')
val_datagen = ImageDataGenerator()

In [4]:
model = unet(input_size = (256,256,3))
model_checkpoint = ModelCheckpoint('ventral.hdf5', monitor='val_loss', verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor="val_loss",verbose = 1,mode='min',patience=70)
reduce_lr =  ReduceLROnPlateau(monitor = "val_loss", factor = 0.5, patience = 10,verbose = 0, mode = "auto", epsilon = 1e-04, cooldown = 0,min_lr = 1e-5)
model.fit_generator(train_generator,steps_per_epoch=iterations_per_epoch,epochs=epochs,callbacks=[model_checkpoint,early_stopping,reduce_lr],validation_data=val_datagen.flow(X_test, Y_test, batch_size=batch_size))

Instructions for updating:
Please use Model.fit, which supports generators.
Found 565 images belonging to 1 classes.
Found 565 images belonging to 1 classes.
Epoch 1/400
300/300 [==============================] - ETA: 0s - loss: 0.7395 - dice_coef: 0.2604 - accuracy: 0.9838
Epoch 00001: val_loss improved from inf to 0.75044, saving model to ventral.hdf5
300/300 [==============================] - 139s 465ms/step - loss: 0.7395 - dice_coef: 0.2604 - accuracy: 0.9838 - val_loss: 0.7504 - val_dice_coef: 0.2532 - val_accuracy: 0.9778
Epoch 2/400
300/300 [==============================] - ETA: 0s - loss: 0.5790 - dice_coef: 0.4213 - accuracy: 0.9867
Epoch 00002: val_loss improved from 0.75044 to 0.73347, saving model to ventral.hdf5
300/300 [==============================] - 137s 455ms/step - loss: 0.5790 - dice_coef: 0.4213 - accuracy: 0.9867 - val_loss: 0.7335 - val_dice_coef: 0.2693 - val_accuracy: 0.9754
Epoch 3/400
300/300 [==============================] - ETA: 0s - loss: 0.4929 - dice

Epoch 22/400
300/300 [==============================] - ETA: 0s - loss: 0.3798 - dice_coef: 0.6202 - accuracy: 0.9970
Epoch 00022: val_loss did not improve from 0.67633
300/300 [==============================] - 136s 454ms/step - loss: 0.3798 - dice_coef: 0.6202 - accuracy: 0.9970 - val_loss: 0.7627 - val_dice_coef: 0.2365 - val_accuracy: 0.9869
Epoch 23/400
300/300 [==============================] - ETA: 0s - loss: 0.3804 - dice_coef: 0.6197 - accuracy: 0.9969
Epoch 00023: val_loss did not improve from 0.67633
300/300 [==============================] - 136s 453ms/step - loss: 0.3804 - dice_coef: 0.6197 - accuracy: 0.9969 - val_loss: 0.8465 - val_dice_coef: 0.1515 - val_accuracy: 0.9862
Epoch 24/400
300/300 [==============================] - ETA: 0s - loss: 0.3777 - dice_coef: 0.6223 - accuracy: 0.9970
Epoch 00024: val_loss improved from 0.67633 to 0.58855, saving model to ventral.hdf5
300/300 [==============================] - 137s 456ms/step - loss: 0.3777 - dice_coef: 0.6223 - accur

300/300 [==============================] - ETA: 0s - loss: 0.3586 - dice_coef: 0.6415 - accuracy: 0.9973
Epoch 00045: val_loss did not improve from 0.55190
300/300 [==============================] - 136s 453ms/step - loss: 0.3586 - dice_coef: 0.6415 - accuracy: 0.9973 - val_loss: 0.5744 - val_dice_coef: 0.4270 - val_accuracy: 0.9889
Epoch 46/400
300/300 [==============================] - ETA: 0s - loss: 0.3582 - dice_coef: 0.6419 - accuracy: 0.9973
Epoch 00046: val_loss did not improve from 0.55190
300/300 [==============================] - 136s 453ms/step - loss: 0.3582 - dice_coef: 0.6419 - accuracy: 0.9973 - val_loss: 0.5590 - val_dice_coef: 0.4424 - val_accuracy: 0.9896
Epoch 47/400
300/300 [==============================] - ETA: 0s - loss: 0.3573 - dice_coef: 0.6428 - accuracy: 0.9973
Epoch 00047: val_loss did not improve from 0.55190
300/300 [==============================] - 136s 454ms/step - loss: 0.3573 - dice_coef: 0.6428 - accuracy: 0.9973 - val_loss: 0.6545 - val_dice_coef:

Epoch 69/400
300/300 [==============================] - ETA: 0s - loss: 0.3420 - dice_coef: 0.6581 - accuracy: 0.9975
Epoch 00069: val_loss did not improve from 0.55190
300/300 [==============================] - 136s 453ms/step - loss: 0.3420 - dice_coef: 0.6581 - accuracy: 0.9975 - val_loss: 0.7560 - val_dice_coef: 0.2434 - val_accuracy: 0.9870
Epoch 70/400
300/300 [==============================] - ETA: 0s - loss: 0.3409 - dice_coef: 0.6589 - accuracy: 0.9975
Epoch 00070: val_loss did not improve from 0.55190
300/300 [==============================] - 136s 453ms/step - loss: 0.3409 - dice_coef: 0.6589 - accuracy: 0.9975 - val_loss: 0.6843 - val_dice_coef: 0.3115 - val_accuracy: 0.9879
Epoch 71/400
300/300 [==============================] - ETA: 0s - loss: 0.3416 - dice_coef: 0.6581 - accuracy: 0.9974
Epoch 00071: val_loss did not improve from 0.55190
300/300 [==============================] - 136s 453ms/step - loss: 0.3416 - dice_coef: 0.6581 - accuracy: 0.9974 - val_loss: 0.6531 - v

Epoch 93/400
300/300 [==============================] - ETA: 0s - loss: 0.3193 - dice_coef: 0.6807 - accuracy: 0.9978
Epoch 00093: val_loss did not improve from 0.55190
300/300 [==============================] - 136s 453ms/step - loss: 0.3193 - dice_coef: 0.6807 - accuracy: 0.9978 - val_loss: 0.6473 - val_dice_coef: 0.3486 - val_accuracy: 0.9882
Epoch 94/400
300/300 [==============================] - ETA: 0s - loss: 0.3187 - dice_coef: 0.6813 - accuracy: 0.9978
Epoch 00094: val_loss did not improve from 0.55190
300/300 [==============================] - 136s 453ms/step - loss: 0.3187 - dice_coef: 0.6813 - accuracy: 0.9978 - val_loss: 0.8125 - val_dice_coef: 0.1891 - val_accuracy: 0.9865
Epoch 95/400
300/300 [==============================] - ETA: 0s - loss: 0.3176 - dice_coef: 0.6825 - accuracy: 0.9978
Epoch 00095: val_loss did not improve from 0.55190
300/300 [==============================] - 136s 452ms/step - loss: 0.3176 - dice_coef: 0.6825 - accuracy: 0.9978 - val_loss: 0.7781 - v

...

300/300 [==============================] - ETA: 0s - loss: 0.2199 - dice_coef: 0.7802 - accuracy: 0.9987
Epoch 00258: val_loss did not improve from 0.50317
300/300 [==============================] - 136s 453ms/step - loss: 0.2199 - dice_coef: 0.7802 - accuracy: 0.9987 - val_loss: 0.5285 - val_dice_coef: 0.4748 - val_accuracy: 0.9885
Epoch 259/400
300/300 [==============================] - ETA: 0s - loss: 0.2192 - dice_coef: 0.7807 - accuracy: 0.9987
Epoch 00259: val_loss did not improve from 0.50317
300/300 [==============================] - 136s 452ms/step - loss: 0.2192 - dice_coef: 0.7807 - accuracy: 0.9987 - val_loss: 0.5459 - val_dice_coef: 0.4533 - val_accuracy: 0.9873
Epoch 260/400
300/300 [==============================] - ETA: 0s - loss: 0.2188 - dice_coef: 0.7812 - accuracy: 0.9987
Epoch 00260: val_loss did not improve from 0.50317
300/300 [==============================] - 136s 453ms/step - loss: 0.2188 - dice_coef: 0.7812 - accuracy: 0.9987 - val_loss: 0.5683 - val_dice_coe

Epoch 282/400
300/300 [==============================] - ETA: 0s - loss: 0.2069 - dice_coef: 0.7930 - accuracy: 0.9988
Epoch 00282: val_loss did not improve from 0.49977
300/300 [==============================] - 136s 452ms/step - loss: 0.2069 - dice_coef: 0.7930 - accuracy: 0.9988 - val_loss: 0.5221 - val_dice_coef: 0.4792 - val_accuracy: 0.9888
Epoch 283/400
300/300 [==============================] - ETA: 0s - loss: 0.2066 - dice_coef: 0.7934 - accuracy: 0.9988
Epoch 00283: val_loss did not improve from 0.49977
300/300 [==============================] - 136s 453ms/step - loss: 0.2066 - dice_coef: 0.7934 - accuracy: 0.9988 - val_loss: 0.5462 - val_dice_coef: 0.4532 - val_accuracy: 0.9881
Epoch 284/400
300/300 [==============================] - ETA: 0s - loss: 0.2061 - dice_coef: 0.7938 - accuracy: 0.9988
Epoch 00284: val_loss did not improve from 0.49977
300/300 [==============================] - 136s 452ms/step - loss: 0.2061 - dice_coef: 0.7938 - accuracy: 0.9988 - val_loss: 0.5286 

Epoch 306/400
300/300 [==============================] - ETA: 0s - loss: 0.1951 - dice_coef: 0.8049 - accuracy: 0.9989
Epoch 00306: val_loss did not improve from 0.49977
300/300 [==============================] - 136s 452ms/step - loss: 0.1951 - dice_coef: 0.8049 - accuracy: 0.9989 - val_loss: 0.5383 - val_dice_coef: 0.4574 - val_accuracy: 0.9870
Epoch 307/400
300/300 [==============================] - ETA: 0s - loss: 0.1944 - dice_coef: 0.8057 - accuracy: 0.9989
Epoch 00307: val_loss did not improve from 0.49977
300/300 [==============================] - 136s 452ms/step - loss: 0.1944 - dice_coef: 0.8057 - accuracy: 0.9989 - val_loss: 0.5016 - val_dice_coef: 0.4950 - val_accuracy: 0.9889
Epoch 308/400
300/300 [==============================] - ETA: 0s - loss: 0.1941 - dice_coef: 0.8058 - accuracy: 0.9989
Epoch 00308: val_loss did not improve from 0.49977
300/300 [==============================] - 135s 452ms/step - loss: 0.1941 - dice_coef: 0.8058 - accuracy: 0.9989 - val_loss: 0.5279 

Epoch 330/400
300/300 [==============================] - ETA: 0s - loss: 0.1839 - dice_coef: 0.8160 - accuracy: 0.9989
Epoch 00330: val_loss did not improve from 0.49977
300/300 [==============================] - 137s 457ms/step - loss: 0.1839 - dice_coef: 0.8160 - accuracy: 0.9989 - val_loss: 0.5051 - val_dice_coef: 0.4991 - val_accuracy: 0.9893
Epoch 331/400
300/300 [==============================] - ETA: 0s - loss: 0.1831 - dice_coef: 0.8169 - accuracy: 0.9990
Epoch 00331: val_loss did not improve from 0.49977
300/300 [==============================] - 137s 456ms/step - loss: 0.1831 - dice_coef: 0.8169 - accuracy: 0.9990 - val_loss: 0.5166 - val_dice_coef: 0.4829 - val_accuracy: 0.9888
Epoch 332/400
300/300 [==============================] - ETA: 0s - loss: 0.1828 - dice_coef: 0.8172 - accuracy: 0.9990
Epoch 00332: val_loss did not improve from 0.49977
300/300 [==============================] - 137s 457ms/step - loss: 0.1828 - dice_coef: 0.8172 - accuracy: 0.9990 - val_loss: 0.5108 

KeyboardInterrupt: 

In [124]:
import sys
from PIL import Image

# Create side by side png
i=0
for filepath in os.listdir('../data/test_images/ventral_samples_R0004'):
    image = Image.open('../data/test_images/ventral_samples_R0004/'+filepath)
    image = image.resize((256, 256))
    # convert image to numpy array
    data = np.asarray(image)
    data = data/255.
    arr = (model.predict(data.reshape(1,256,256,3))[0]).reshape(256,256)
    mask = Image.fromarray((arr*255))
    new_im = Image.new('RGB', (512, 256))
    new_im.paste(mask, (0,0))
    new_im.paste(image,(256,0))
    new_im.save('vid/%d.png'%(i))
    i=i+1